In [6]:
#Importing packages
from selenium import webdriver
import pandas as pd

In [65]:
#change directory where you put your chrome driver
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome("chromedriver", options=options)

In [67]:
%%time
all_comments = pd.DataFrame(columns = ['Date','user_id', 'comments']) 

x = 334
pages_scraped = 434
while x <= pages_scraped:
    print("page",x)
    
    if x > 1:
        driver.find_element_by_link_text("{}".format(x)).click()
        driver.implicitly_wait(3)
    
    ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('id'))

    for d in comment_ids:

        #Extract dates from for each user on a page
        user_date = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
        date = user_date.get_attribute('title')

        #Extract user ids from each user on a page
        userid_element = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
        userid = userid_element.text

        #Extract Message for each user on a page
        if driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/p'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/p')[0]
            comment = user_message.text
        elif driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/ul/li'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]/ul/li')[0]
            comment = user_message.text
        elif driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]'):
            user_message = driver.find_elements_by_xpath('//*[@id="' + d +'"]/div/div[3]/div/div[1]')[0]
            comment = user_message.text
        else: 
            comment = None
        
        #Adding date, userid and comment for each user in a dataframe    
        all_comments.loc[len(all_comments)] = [date,userid,comment]
        
    x += 1

page 334
page 335
page 336
page 337
page 338
page 339
page 340
page 341
page 342
page 343
page 344
page 345
page 346
page 347
page 348
page 349
page 350
page 351
page 352
page 353
page 354
page 355
page 356
page 357
page 358
page 359
page 360
page 361
page 362
page 363
page 364
page 365
page 366
page 367
page 368
page 369
page 370
page 371
page 372
page 373
page 374
page 375
page 376
page 377
page 378
page 379
page 380
page 381
page 382
page 383
page 384
page 385
page 386
page 387
page 388
page 389
page 390
page 391
page 392
page 393
page 394
page 395
page 396
page 397
page 398
page 399
page 400
page 401
page 402
page 403
page 404
page 405
page 406
page 407
page 408
page 409
page 410
page 411
page 412
page 413
page 414
page 415
page 416
page 417
page 418
page 419
page 420
page 421
page 422
page 423
page 424
page 425
page 426
page 427
page 428
page 429
page 430
page 431
page 432
page 433
page 434
Wall time: 7h 26min 3s


In [68]:
all_comments.shape

(5014, 3)

In [69]:
# all_comments.to_csv("5000_comments_edmunds.csv", sep="\t")

#### check the data

In [76]:
import pandas as pd
df = pd.read_csv("5000_comments_edmunds.csv", '\t', parse_dates=['Date'])

In [77]:
len(df['user_id'].value_counts())

68

In [79]:
df['Date'].min()

Timestamp('2014-02-20 14:06:00')

#### Start with Sentence